In [6]:
import sys

%pip install --upgrade pip setuptools wheel
%pip install "numpy<2.0" scipy pandas scikit-learn matplotlib wordcloud tqdm
%pip install umap-learn hdbscan
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
%pip install sentence-transformers transformers bertopic
%pip install plotly ipywidgets
%pip -q install openpyxl

import numpy, pandas, sklearn, matplotlib, wordcloud, umap, hdbscan, torch, openpyxl
import transformers, sentence_transformers, bertopic


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
#This example illustrates how to instantiate a Latent Semantic Analysis Topic Model
#in Python and provides a few different ways to look at the results

#The preprocessing of the data (a relatively simple and small corpus of n = 250)
#is not necessarily optimized to the data

#This examples uses TF-IDF (a frequency-based method) for word embedding
#TF-IDF is also not necessarily optimal for word embedding this data

#Import all necessary modules and packages to perform LSA
#For creating dataframes and handling arrays
import pandas as pd #enables use of pandas libraries with alias pd
import numpy as np #enables use of numpy libraries with alias np
import string

#For removing stopwords
# from nltk.corpus import stopwords

#For word embedding using frequency/counts method
from sklearn.feature_extraction.text import CountVectorizer

#For plotting
import matplotlib.pyplot as plt

#For word clouds
from wordcloud import WordCloud

#from word embedding using TF-IDF methods
from sklearn.feature_extraction.text import TfidfVectorizer

#import LSA modelling
from sklearn.decomposition import TruncatedSVD

#For calculating Umass topic coherence
import math
from collections import defaultdict

In [8]:

#from google.colab import drive
#drive.mount('/content/drive')

#This examples uses the Peer Support Level 1 Dataset located in the Datasets Module on Canvas
# Load dataset

# **IMPORTANT make sure dataset is uploaded to your google drive, copy file path using the folder file on the left
# file_path = '/Users/HocVuu/Desktop/UW School Folder/ee400PeerSupport_Level1.xlsx'
file_path = '/Users/HocVuu/Desktop/UW School Folder/ee400/PeerSupport_Level2.xlsx'

# file_path = r"C:\Users\falco\Downloads\PeerSupport_Level2.xlsx"
data = pd.read_excel(file_path)
#print (data)

data_for_nmf = data[['PeerSupportResponse']].copy()

# Convert all elements in data_for_lda_df into strings
#data_for_lda_df = data_for_lda_df.astype(str)
print (data_for_nmf['PeerSupportResponse'])

0      Not wast etime by going around in circles with...
1                                     Ask good questions
2      Reduce the amount of talking and other distrac...
3      Always ask questions if they don't understand....
4                               Not watch anime in class
                             ...                        
797    make sure to keep questions on topic rather th...
798                 ask lots of questions during lecture
799    Ask more questions clarifying the topics first...
800                                  Ask good questions.
801                                                     
Name: PeerSupportResponse, Length: 802, dtype: object


In [9]:
#Preprocess data_for_kmeans by converting all text to lower case

# Convert all text to lower case
#data_for_lda_df.loc[:, 'LowerCase'] = data_for_lda_df.str.lower()
temp = data_for_nmf['PeerSupportResponse'].str.lower()
data_for_nmf.loc[:, 'LowerCase'] = temp

# Look at resulting text to confirm that conversion was done correctly
print (data_for_nmf)

                                   PeerSupportResponse  \
0    Not wast etime by going around in circles with...   
1                                   Ask good questions   
2    Reduce the amount of talking and other distrac...   
3    Always ask questions if they don't understand....   
4                             Not watch anime in class   
..                                                 ...   
797  make sure to keep questions on topic rather th...   
798               ask lots of questions during lecture   
799  Ask more questions clarifying the topics first...   
800                                Ask good questions.   
801                                                      

                                             LowerCase  
0    not wast etime by going around in circles with...  
1                                   ask good questions  
2    reduce the amount of talking and other distrac...  
3    always ask questions if they don't understand....  
4                 

In [10]:
# ===== Faux lemmatization + custom stopwords =====
from wordcloud import STOPWORDS

# 1) Start from the original text, lower-case and basic cleanup
data_for_nmf["LowerCase"] = (
    data_for_nmf["PeerSupportResponse"].astype(str).str.lower()
    .str.replace(r"[’']", "'", regex=True)
    .str.replace(r"[^a-z0-9\s]", " ", regex=True)   # drop punctuation
    .str.replace(r"\s+", " ", regex=True)           # collapse spaces
    .str.strip()
)

# 2) Faux lemmatize with word-boundary replacements (avoid partial matches)
s = data_for_nmf["LowerCase"]
repls = {
    r"\bquestions\b":   "question",
    r"\basking\b":      "ask",
    r"\btalking\b":     "talk",
    r"\bstudents\b":    "student",
    r"\bdistracting\b": "distract",
    r"\blectures\b":    "lecture",
    r"\bprofessors\b":  "professor",
}
for pat, rep in repls.items():
    s = s.str.replace(pat, rep, regex=True)
data_for_nmf["LowerCase"] = s

# 3) Build stopword set and remove stopwords
custom = {"student", "professor", "lecture"}
stop = set(STOPWORDS) | custom

data_for_nmf["NoStopWords"] = (
    data_for_nmf["LowerCase"].fillna("")
    .apply(lambda x: " ".join(w for w in x.split() if w not in stop))
)

In [11]:
#Remove punctuation from dataset

def remove_punctuation(text):
    """Removes punctuation from a string."""
    return ''.join([char for char in text if char not in string.punctuation])

data_for_nmf['RemovePunctuation'] = data_for_nmf['NoStopWords'].apply(remove_punctuation)
print (data_for_nmf)

                                   PeerSupportResponse  \
0    Not wast etime by going around in circles with...   
1                                   Ask good questions   
2    Reduce the amount of talking and other distrac...   
3    Always ask questions if they don't understand....   
4                             Not watch anime in class   
..                                                 ...   
797  make sure to keep questions on topic rather th...   
798               ask lots of questions during lecture   
799  Ask more questions clarifying the topics first...   
800                                Ask good questions.   
801                                                      

                                             LowerCase  \
0    not wast etime by going around in circles with...   
1                                    ask good question   
2    reduce the amount of talk and other distractio...   
3    always ask question if they don t understand c...   
4            

In [12]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

docs = data_for_nmf['RemovePunctuation'].astype(str).tolist()

# CountVectorizer (BERTopic uses c-TF-IDF)
vectorizer_model = CountVectorizer(
    stop_words="english",
    ngram_range=(3, 3),
    min_df=2,
    max_df=0.90
)

# Defaults: UMAP + HDBSCAN + MiniLM embeddings; enable probabilities for distribution plots
topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    calculate_probabilities=True,
    verbose=True
)

# Fit
topics, probabilities = topic_model.fit_transform(docs)


2025-11-06 11:41:58,029 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

2025-11-06 11:42:11,221 - BERTopic - Embedding - Completed ✓
2025-11-06 11:42:11,222 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-06 11:42:16,179 - BERTopic - Dimensionality - Completed ✓
2025-11-06 11:42:16,180 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-06 11:42:16,220 - BERTopic - Cluster - Completed ✓
2025-11-06 11:42:16,224 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-06 11:42:16,241 - BERTopic - Representation - Completed ✓


In [ ]:
# Topic overview
topic_info = topic_model.get_topic_info()
topic_info.head(10)



,Topic,Count,Name,Representation,Representative_Docs
0,-1,183,-1_ask question chat_improve educational exper...,"[ask question chat, improve educational experi...",[ask question chat helps others understand mat...
1,0,111,0_ask good question_good question ask_ask ques...,"[ask good question, good question ask, ask que...","[ask good question, ask question sometimes m c..."
2,1,77,1_ask question ask_question ask question_ask q...,"[ask question ask, question ask question, ask ...","[ask question interested answers, ask question..."
3,2,59,2_question class ask_ask question class_class ...,"[question class ask, ask question class, class...","[feel embarassed ask question class, ask quest..."
4,3,58,3_classes improve educational_improve educatio...,"[classes improve educational, improve educatio...",[classes improve educational experience ask cl...
5,4,50,4_question making sure_question make sure_don ...,"[question making sure, question make sure, don...","[ensure mics unless ask question, make sure mi..."
6,5,36,5_talk distract quiet_class improve educationa...,"[talk distract quiet, class improve educationa...",[quiet teacher talk collaborate neighbors ask ...
7,6,32,6_use chat box_don distract zoom_comfortable a...,"[use chat box, don distract zoom, comfortable ...","[need use chat box educational purposes, use c..."
8,7,30,7____,"[, , , , , , , , , ]","[respect, respectful question responses, liste..."
9,8,25,8_educational experience respectful_don afraid...,"[educational experience respectful, don afraid...","[stay quiet lecturer speaking speak question, ..."


In [ ]:
# Export top x topics with counts and some documents
from google.colab import files

# CHOOSE YOUR OWN
out_path = r"C:\Users\stlp\Desktop\topic_info_top10.xlsx"
topic_info.head(10).to_excel(out_path, index=False, engine="openpyxl")
files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Excel sheet of all documents and assigned topics
try:
    doc_info = topic_model.get_document_info(docs)
except TypeError:
    doc_info = topic_model.get_document_info()

out = doc_info[["Topic", "Document"]]

# CHOOSE YOUR OWN
out_path1 = r"C:\Users\stlp\Desktop\topic_document_all.xlsx"
out.to_excel(out_path1, index=False, engine="openpyxl")
files.download(out_path1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Visualize Topics (2D topic map)
fig_topics = topic_model.visualize_topics()
fig_topics.show()
# fig_topics.write_html("bertopic_topics.html")


In [ ]:
# Topic Similarity (Heatmap)
# reorder topics into clusters for readability
fig_heat = topic_model.visualize_heatmap(n_clusters=16)
fig_heat.show()
# fig_heat.write_html("bertopic_similarity_heatmap.html")


In [ ]:
# Visualize Documents (Plotly scatter)
fig_docs = topic_model.visualize_documents(docs)
fig_docs.show()
# fig_docs.write_html("bertopic_documents.html")


In [ ]:
# Probabilities / Distribution (per-document topic mix)
# Show distribution for the first document (change index as needed)
fig_dist = topic_model.visualize_distribution(probabilities[0])
fig_dist.show()
# fig_dist.write_html("bertopic_distribution_doc0.html")


In [ ]:
# Hierarchical labels + hierarchical visualization
# Generate readable labels from top words
labels = topic_model.generate_topic_labels(
    nr_words=3,          # tweak if you want longer labels
    topic_prefix=False,
    separator=", "
)
topic_model.set_topic_labels(labels)

#  Compute a topic hierarchy and plot
hier_df = topic_model.hierarchical_topics(docs)
fig_hier = topic_model.visualize_hierarchy(custom_labels=True)  # uses labels set above
fig_hier.show()
# fig_hier.write_html("bertopic_hierarchy.html")



100%|██████████| 16/16 [00:00<00:00, 245.27it/s]


In [ ]:
# Top words per topic (for slide tables)
topic_model.get_topic(0)
topic_model.get_topics()

# Topic frequencies
topic_model.get_topic_freq().head(10)

,Topic,Count
2,-1,183
0,0,111
3,1,77
7,2,59
5,3,58
15,4,50
1,5,36
16,6,32
12,7,30
6,8,25
